Libraries and options

In [1]:
import numpy as np, pandas as pd, sys, os, re, matplotlib.pyplot as plt, seaborn as sns,time
import json, shutil, zipfile, bz2, pickle
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500
pd.options.display.max_colwidth = 50

In [2]:
def get_counties_df(file_name='../covid/config/counties.json'):
    with open(file_name) as file:
                county_data = json.load(file)
    
    df = pd.DataFrame.from_dict(county_data)
    
    data = [df[col] for col in df.columns]    
    
    # pivot
    return pd.DataFrame(data,columns=df.index, index=df.columns)
    
counties = get_counties_df()  

In [3]:
# counties['fips'] = counties['fips']['id'].astype(str)

a = [x['id'] for x in counties['fips'][~counties['fips'].isnull()] ]
counties.loc[counties['fips'].notnull(),'fips'] = a

Importing census data

In [4]:
censusDF = pd.read_csv('./censusData.csv')

censusDF['fips'] = censusDF['fips'].astype('str').str.zfill(5)
censusDF['state']  = censusDF['state'].astype('str').str.zfill(2)
censusDF['county'] = censusDF['county'].astype('str').str.zfill(3)

In [5]:
# _=[print('###',i,j) for i,j in enumerate(censusDF.columns)]
cols=['totalMoved','movedWithinState','movedWithoutState','movedFromAbroad','publicTrans','totalTrans','householdsTotal',
      'houseWith65','house2+with65','houseFamily65','houseNonfam65','houseNo65',
      'house2+No65','houseFamilyNo65','houseNonfamNo65','householdStructuresTotal','householdIncomeMedian',
      'gini','hoursWorkedMean','unitsInStructure','healthInsTotal','healthInsNativeWith',
      'healthInsForeignNatWith','healthInsForeignNoncitWith','healthInsForeignNatNo','healthInsForeignNoncitNo','healthInsNativeNo',
      'name','state','county','fips'
     ]
censusDF = pd.DataFrame(censusDF.values, columns=cols)

In [6]:
a = counties.index
x = [re.search('.*(?=_UnitedStates)',x).group(0).split('_') for x in a]
x,y = zip(*x)
counties['coName'] = x
counties['stName'] = y

In [7]:
def cenCounty( c ):
    toby =  []
    for x in c:
        try:
            toby.append(re.search('.*(?=\sCounty|\sCity\sand\sBorough|(?<!and)\sBorough|\sCensus|\sMunicipality)',x).group(0))
        except:
            toby.append(re.search('.*(?=,\s)',x).group(0))
    return toby

def cenState( c ):
    toby =  []
    for x in c:
        try:
            toby.append(re.search('(?<=,\s).*$',x).group(0))
        except:
            print('problem',x)
            raise Exception('What the hell???')
    return toby

def addSpace( df,col ):
    vals = df[col].values
    return [re.sub('(?<=[a-z]|\.)(?=[A-Z])',' ',x) for x in vals]

In [8]:
a =  censusDF.name.values
x = cenCounty(a)
censusDF['coName'] =  x
y = cenState(a)
censusDF['stName'] = y
censusDF[['name','coName','stName','fips']]

# [counties[x].index for x in counties.index if re.match('Unassigned_',x)]
# biggy = []
# for x in counties.index:
#     if re.match('Unassigned_|Outof',x):
#         biggy.append(x)
# counties.drop(biggy,inplace=True)
# counties.tail()

,name,coName,stName,fips
0,"Fayette County, Illinois",Fayette,Illinois,17051
1,"Logan County, Illinois",Logan,Illinois,17107
2,"Saline County, Illinois",Saline,Illinois,17165
3,"Lake County, Illinois",Lake,Illinois,17097
4,"Massac County, Illinois",Massac,Illinois,17127
...,...,...,...,...
3215,"Crockett County, Tennessee",Crockett,Tennessee,47033
3216,"Lake County, Tennessee",Lake,Tennessee,47095
3217,"Knox County, Tennessee",Knox,Tennessee,47093
3218,"Benton County, Washington",Benton,Washington,53005


Importing covid data

In [9]:
path = r'../covid/processed_data/county_merged_parts/'
listFiles = []
for r, d, files in os.walk(path):
    for file in files:
        listFiles.append(file)

testDF = pd.DataFrame()
for i,file in enumerate(listFiles):
    inDF = pd.read_pickle(f'{path}{file}')
    inDF['bigName'] = re.search('.*(?=.pkl)',file).group(0)
    inDF.reset_index(level=0,inplace=True)
    testDF = pd.concat([testDF,inDF],ignore_index=True)

In [10]:
def getCounty( c ):
    toby =  []
    for x in c:
        try:
            toby.append(re.search('^.*(?=_(?!United))',x).group(0))
        except:
            print('problem',x)
    return toby

def getState( c ):
    toby =  []
    for x in c:
        try:
            toby.append(re.search('(?<=_).*(?=.UnitedStates)',x).group(0))
        except:
            print('problem',x)
    return toby    

def addSpace( df,col ):
    vals = df[col].values
    return [re.sub('(?<=[a-z]|\.)(?=[A-Z])',' ',x) for x in vals]

In [11]:
a =  testDF.bigName.values
x = getCounty(a)
testDF['coName'] =  x
testDF['coName'] =  addSpace(testDF,'coName')
y = getState(a)
y
testDF['stName'] = y
testDF['stName'] = addSpace(testDF,'stName')
testDF[['bigName','coName','stName','fips']]

,bigName,coName,stName,fips
0,Abbeville_SouthCarolina_UnitedStates,Abbeville,South Carolina,45001.0
1,Abbeville_SouthCarolina_UnitedStates,Abbeville,South Carolina,45001.0
2,Abbeville_SouthCarolina_UnitedStates,Abbeville,South Carolina,45001.0
3,Abbeville_SouthCarolina_UnitedStates,Abbeville,South Carolina,45001.0
4,Abbeville_SouthCarolina_UnitedStates,Abbeville,South Carolina,45001.0
...,...,...,...,...
2122225,Ziebach_SouthDakota_UnitedStates,Ziebach,South Dakota,46137.0
2122226,Ziebach_SouthDakota_UnitedStates,Ziebach,South Dakota,46137.0
2122227,Ziebach_SouthDakota_UnitedStates,Ziebach,South Dakota,46137.0
2122228,Ziebach_SouthDakota_UnitedStates,Ziebach,South Dakota,46137.0


In [12]:
testDF['intfips'] = testDF['fips'].fillna(0).astype(int)
testDF['intfips']= testDF['intfips'].round()
testDF['strfips'] = testDF['intfips'].astype(str)
testDF['fips'] =testDF['strfips']
testDF['fips'].where(testDF['fips']!='0',inplace=True)
testDF.drop(columns=['intfips','strfips'],inplace=True)
testDF['fips'] = testDF['fips'].str.zfill(5)
testDF['fips'].dtype

dtype('O')

In [13]:
testDF['dates'] = pd.to_datetime(testDF['dates'])

In [14]:
mask = testDF['fips'].isnull()
indices = testDF[mask].index
noFips = testDF.loc[indices,['coName','stName']]
noFips['index'] = noFips.index
test = noFips.merge(censusDF[['coName','stName','fips']],left_on=['coName','stName'],right_on=['coName','stName'],how='left')
test.set_index(test['index'],inplace=True)
test['coName'][test['fips'].isnull()].unique()

array(['Dukesand Nantucket', 'Falls Church', 'Kansas City',
       'Kodiak Island', 'Lakeand Peninsula', 'Lakeofthe Woods',
       'La Moure', 'Martinsville', 'Mc Cone', 'Mc Mullen',
       'New York City', 'Norton', 'Outof AK', 'Outof AL', 'Outof AR',
       'Outof AZ', 'Outof CA', 'Outof CO', 'Outof CT', 'Outof DC',
       'Outof DE', 'Outof FL', 'Outof GA', 'Outof HI', 'Outof IA',
       'Outof ID', 'Outof IL', 'Outof IN', 'Outof KS', 'Outof KY',
       'Outof LA', 'Outof MA', 'Outof MD', 'Outof ME', 'Outof MI',
       'Outof MN', 'Outof MO', 'Outof MS', 'Outof MT', 'Outof NC',
       'Outof ND', 'Outof NE', 'Outof NH', 'Outof NJ', 'Outof NM',
       'Outof NV', 'Outof NY', 'Outof OH', 'Outof OK', 'Outof OR',
       'Outof PA', 'Outof RI', 'Outof SC', 'Outof SD', 'Outof TN',
       'Outof TX', 'Outof UT', 'Outof VA', 'Outof VT', 'Outof WA',
       'Outof WI', 'Outof WV', 'Outof WY', 'Princeof Wales-Hyder',
       'St. Bernard', 'St. Charles', 'St. Helena', 'St. James',
       'St. J

In [15]:
testDF.loc[testDF['fips'].isnull(),'fips'] = test['fips']
testDF.dropna(axis=0,subset=['fips'],inplace=True)
# testDF[testDF['coName']=='St. Clair']

In [16]:
print(testDF.shape)
mask = testDF.isnull().any(axis=0)
noNullDF = testDF.loc[:,~mask]
print(noNullDF.shape)
testDF[testDF['fips'].isnull()]

(2038518, 66)
(2038518, 29)


,dates,JHU_ConfirmedCases.data,JHU_ConfirmedCases.missing,NYT_ConfirmedCases.data,NYT_ConfirmedCases.missing,JHU_ConfirmedDeaths.data,JHU_ConfirmedDeaths.missing,JHU_ConfirmedRecoveries.data,JHU_ConfirmedRecoveries.missing,NYT_AllCausesDeathsWeekly_Expected_Deaths_AllCauses.data,NYT_AllCausesDeathsWeekly_Expected_Deaths_AllCauses.missing,NYT_ConfirmedDeaths.data,NYT_ConfirmedDeaths.missing,NYT_AllCausesDeathsWeekly_Excess_Deaths.data,NYT_AllCausesDeathsWeekly_Excess_Deaths.missing,NYT_AllCausesDeathsWeekly_Deaths_AllCauses.data,NYT_AllCausesDeathsWeekly_Deaths_AllCauses.missing,NYT_AllCausesDeathsMonthly_Deaths_AllCauses.data,NYT_AllCausesDeathsMonthly_Deaths_AllCauses.missing,NYT_AllCausesDeathsMonthly_Excess_Deaths.data,NYT_AllCausesDeathsMonthly_Excess_Deaths.missing,NYT_AllCausesDeathsMonthly_Expected_Deaths_AllCauses.data,NYT_AllCausesDeathsMonthly_Expected_Deaths_AllCauses.missing,TotalPopulation.data,TotalPopulation.missing,MaleAndFemale_AtLeast65_Population.data,MaleAndFemale_AtLeast65_Population.missing,Male_Total_Population.data,Male_Total_Population.missing,Female_Total_Population.data,Female_Total_Population.missing,MaleAndFemale_Under18_Population.data,MaleAndFemale_Under18_Population.missing,BLS_EmployedPopulation.data,BLS_EmployedPopulation.missing,BLS_UnemployedPopulation.data,BLS_UnemployedPopulation.missing,BLS_UnemploymentRate.data,BLS_UnemploymentRate.missing,BLS_LaborForcePopulation.data,BLS_LaborForcePopulation.missing,AverageDailyTemperature.data,AverageDailyTemperature.missing,AverageDewPoint.data,AverageDewPoint.missing,AverageRelativeHumidity.data,AverageRelativeHumidity.missing,AverageSurfaceAirPressure.data,AverageSurfaceAirPressure.missing,AveragePrecipitationTotal.data,AveragePrecipitationTotal.missing,AveragePrecipitation.data,AveragePrecipitation.missing,AverageWindDirection.data,AverageWindDirection.missing,AverageWindSpeed.data,AverageWindSpeed.missing,hospitalIcuBeds,hospitalStaffedBeds,hospitalLicensedBeds,latestTotalPopulation,fips,LND110210,bigName,coName,stName


In [17]:
# path = r'../covid/processed_data/county_merged_parts/'
# listFiles = []
# for r, d, files in os.walk(path):
#     for file in files:
#         listFiles.append(file)

# testDF = pd.DataFrame()
# for i,file in enumerate(listFiles):
#     inDF = pd.read_pickle(f'{path}{file}')
#     inDF.reset_index(level=0,inplace=True)
#     testDF = pd.concat([testDF,inDF],ignore_index=True)
# # test = f'{path}{listFiles[0]}'
# # testDF= pd.read_pickle(test)

# # mask = testDF['fips'].isnull()
# testDF.dropna(subset=['fips'], inplace=True)

# testDF['fips']=testDF['fips'].astype(int).astype(str)

# testDF['fips'] = testDF['fips'].str.zfill(5)

In [ ]:
testDF.sort_values(['fips','dates'])['fips']

In [18]:
tt = testDF.groupby(['fips']).rolling(on='dates',window=7).mean()

In [19]:
tt = tt.reset_index()

In [20]:
tt = tt.set_index('level_1')

In [21]:
tt[['LND110210','bigName','coName','stName','ddates']]  = testDF[['LND110210','bigName','coName','stName','dates']]

In [10]:
def exportPbz2( outfile, infile ):
    start = time.time()
    with bz2.BZ2File(outfile+'.pbz2','wb') as file:
        pickle.dump(infile,file)
    print(time.time()-start)
    
def importPbz2( file ):
    start = time.time()
    data = bz2.BZ2File(file,'rb')
    return pd.read_pickle(data)

In [26]:
exportPbz2('covid',tt)

99.7743284702301


In [14]:
tt = importPbz2('covid.pbz2')

Joining census and covid data

In [15]:
newDF = tt.join(censusDF.set_index('fips'),on='fips',how='left',lsuffix='covid',rsuffix='census')

Exported this DF for future work

In [17]:
exportPbz2('covidCensus',newDF)

# newDF.to_csv('covidCensus.gz',index=False,compression='gzip')

139.52572059631348


Imported back

In [11]:
# anotherDF = pd.read_csv('covidCensus.gz')
# anotherDF['fips'] = anotherDF['fips'].astype(str)
# anotherDF['dates']=pd.to_datetime(anotherDF['dates'])
# anotherDF = newDF.copy()
anotherDF = importPbz2('covidCensus.pbz2')

Import air quality data

In [20]:
path = r'../air_quality/'
dirs = next(os.walk(path))[1]
airDF = pd.DataFrame()
cols = []
for dir in dirs:
    files = next(os.walk(f'{path}{dir}/unzipped/'))[2]
    for m,file in enumerate(files):
        inDF = pd.read_csv(f'{path}{dir}/unzipped/{file}')
        cols.append(inDF.columns.values)
        inDF['Date of Last Change'] = pd.to_datetime(inDF['Date of Last Change'])
        inDF['Date Local']= pd.to_datetime(inDF['Date Local'])
        inDF['fips'] = inDF['fips'].astype(str).str.zfill(5)
        prefix = re.search('(?<=daily_)[^_]*(?=_)', f'{file}').group(0)
        print(prefix)
        inDF['pollutant'] = prefix
#         inDF = inDF.add_prefix(prefix+'_')
        airDF =pd.concat([airDF,inDF],ignore_index=True)


lead
lead
lead
no2
no2
no2
ozone
ozone
ozone


C:\Users\richs\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


pm25
pm25
pm25


Getting all unique fips from air quality.

In [21]:
airFips = airDF['fips'].unique()

In [22]:
airDF.sort_values(by=['pollutant','Date Local','fips'],inplace=True)


In [23]:
no2wop = airDF[airDF['pollutant']=='no2'].copy()
leadwop = airDF[airDF['pollutant']=='lead'].copy()
pm25wop = airDF[airDF['pollutant']=='pm25'].copy()
ozonewop = airDF[airDF['pollutant']=='ozone'].copy()

In [24]:
airDF.columns

Index(['State Code', 'County Code', 'Site Num', 'Parameter Code', 'POC',
       'Latitude', 'Longitude', 'Datum', 'Parameter Name', 'Sample Duration',
       'Pollutant Standard', 'Date Local', 'Units of Measure', 'Event Type',
       'Observation Count', 'Observation Percent', 'Arithmetic Mean',
       '1st Max Value', '1st Max Hour', 'AQI', 'Method Code', 'Method Name',
       'Local Site Name', 'Address', 'State Name', 'County Name', 'City Name',
       'CBSA Name', 'Date of Last Change', 'fips', 'pollutant', 'Mean ugm3'],
      dtype='object')

In [25]:
nairDF = airDF.groupby(['pollutant','Date Local','fips']).agg({'Arithmetic Mean':'mean','Mean ugm3':'mean'})

nairDF.reset_index(inplace=True)

In [26]:
polDF = nairDF.merge(airDF, left_on=['Date Local','fips','pollutant'], right_on=['Date Local','fips','pollutant'],how='left')

In [27]:
for a in airDF['pollutant'].unique():
    wob = airDF[['Date Local','Arithmetic Mean']][airDF['pollutant']==a].rolling(window=7,on='Date Local').mean()
    airDF.loc[airDF['pollutant']==a,'rollMeanMean'] = wob['Arithmetic Mean']

In [28]:
wob = airDF[['Date Local','Mean ugm3']][airDF['pollutant']=='no2'].rolling(window=7,on='Date Local').mean()
airDF.loc[airDF['pollutant']=='no2','rollMeanUgm3'] = wob['Mean ugm3']

In [29]:
airDF.loc[airDF['pollutant']=='no2','rollMeanUgm3'].head(20)
# airDF['rollMeanUgm3'].head(20)

23545          NaN
23890          NaN
24240          NaN
24464          NaN
24829          NaN
25189          NaN
25554    17.987273
25919    21.965007
26227    22.384309
26592    18.007714
26938    16.821524
27302    14.689738
27658    14.942643
28023    14.452889
28386    11.749952
28750    13.145258
29112    15.475018
29474    18.264560
29838    20.291058
30198    21.407478
Name: rollMeanUgm3, dtype: float64

In [30]:
exportPbz2('pollution',airDF)

148.17634201049805


In [12]:
airDF = importPbz2('pollution.pbz2')

Merge air and covid data

In [29]:
# tt['ddates']=pd.to_datetime(tt['dates'])
airDF['ddates']=pd.to_datetime(airDF['Date Local'])
# print(tt['ddates'].dtype)
print(airDF['ddates'].dtype)

datetime64[ns]


In [17]:
airFips = airDF['fips'].unique()
print(airFips.size)
# print(tt['fips'].unique().size)
# tt['fips'] = tt['fips'].str.zfill(5)

919


In [30]:
anotherDF['ddates'].dtype

dtype('<M8[ns]')

In [18]:
mask = anotherDF['fips'].isin(airFips)

In [19]:
ttt = anotherDF.loc[mask,:].copy()

In [20]:
ttt[ttt['dates']==ttt['ddates']].shape

(586638, 99)

In [21]:
ttt['ddates'].dtype

dtype('<M8[ns]')

In [31]:
airDF['ddates']

1296      2019-01-01
6247      2019-01-01
6840      2019-01-01
7319      2019-01-01
7860      2019-01-01
             ...    
4565681   2021-04-28
4565700   2021-04-28
4565719   2021-04-28
4565580   2021-04-29
4565581   2021-04-30
Name: ddates, Length: 4584338, dtype: datetime64[ns]

In [32]:
newDF = ttt.merge(airDF,left_on=['fips','ddates'],right_on=['fips','ddates'],how='outer')

In [33]:
newDF['density']=newDF['latestTotalPopulation']/newDF['LND110210']
z = newDF[newDF['dates'].isnull()].index
newDF.loc[z,'dates'] = newDF.loc[z,'ddates']
newDF['dates'] = pd.to_datetime(newDF['dates'])
newDF['caseRate'] = newDF['JHU_ConfirmedCases.data']/newDF['latestTotalPopulation']
newDF['deathRate'] = newDF['JHU_ConfirmedDeaths.data']/newDF['latestTotalPopulation']

In [34]:
_=[print(x) for x in newDF]

fips
dates
AverageDailyTemperature.data
AverageDailyTemperature.missing
AverageDewPoint.data
AverageDewPoint.missing
AveragePrecipitation.data
AveragePrecipitation.missing
AveragePrecipitationTotal.data
AveragePrecipitationTotal.missing
AverageRelativeHumidity.data
AverageRelativeHumidity.missing
AverageSurfaceAirPressure.data
AverageSurfaceAirPressure.missing
AverageWindDirection.data
AverageWindDirection.missing
AverageWindSpeed.data
AverageWindSpeed.missing
BLS_EmployedPopulation.data
BLS_EmployedPopulation.missing
BLS_LaborForcePopulation.data
BLS_LaborForcePopulation.missing
BLS_UnemployedPopulation.data
BLS_UnemployedPopulation.missing
BLS_UnemploymentRate.data
BLS_UnemploymentRate.missing
Female_Total_Population.data
Female_Total_Population.missing
JHU_ConfirmedCases.data
JHU_ConfirmedCases.missing
JHU_ConfirmedDeaths.data
JHU_ConfirmedDeaths.missing
JHU_ConfirmedRecoveries.data
JHU_ConfirmedRecoveries.missing
LND110210
MaleAndFemale_AtLeast65_Population.data
MaleAndFemale_AtLea

Export

In [35]:
exportPbz2('covidPollutionCensus',newDF)

658.7257816791534


In [37]:
anotherDF.to_csv('covidCensus.bz2',index=False,compression='bz2')

##### Below here is historic junk

In [52]:
county_mean= newDF.groupby(['dates','fips']).agg({'rollMeanMean':'mean', 'rollMeanUgm3':'mean'})

In [53]:
county_mean

rollMeanMean  rollMeanUgm3
dates      fips                             
2019-01-01 01049           NaN           NaN
           01073           NaN           NaN
           02068           NaN           NaN
           02090    271.019048           NaN
           04003           NaN           NaN
...                        ...           ...
2021-10-15 56035           NaN           NaN
           56037           NaN           NaN
           56039           NaN           NaN
           56041           NaN           NaN
           56045           NaN           NaN

[856395 rows x 2 columns]

In [54]:
no2wop = newDF[newDF['pollutant']=='no2'].copy()
leadwop = newDF[newDF['pollutant']=='lead'].copy()
pm25wop = newDF[newDF['pollutant']=='pm25'].copy()
ozonewop = newDF[newDF['pollutant']=='ozone'].copy()

In [60]:
wop

rollMeanMean  rollMeanUgm3
dates      fips                             
2019-01-01 01073           NaN           NaN
           02090           NaN           NaN
           04013     11.052587     20.778863
           04019      9.263095     17.414619
           05035      7.813691     14.689738
...                        ...           ...
2021-04-30 19153      5.772294     10.851914
           19177      5.320346     10.002251
           23005      5.629077     10.582664
           40109      6.615873     12.437842
           40143      7.372421     13.860151

[189064 rows x 2 columns]

In [66]:
wop = no2wop.groupby(['dates','fips']).agg({'rollMeanMean':'mean', 'rollMeanUgm3':'mean'})
wop.reset_index().set_index('dates')
# wop.index = pd.to_datetime(wop.index)

,fips,rollMeanMean,rollMeanUgm3
dates,,,
2019-01-01,01073,NaN,NaN
2019-01-01,02090,NaN,NaN
2019-01-01,04013,11.052587,20.778863
2019-01-01,04019,9.263095,17.414619
2019-01-01,05035,7.813691,14.689738
...,...,...,...
2021-04-30,19153,5.772294,10.851914
2021-04-30,19177,5.320346,10.002251
2021-04-30,23005,5.629077,10.582664


In [ ]:
no2wop.sort_values(by=['dates','fips'],inplace=True)

In [68]:
# pm25DF['Date of Last Change']
anotherDF['dates']

level_1
71940     2020-01-01
71941     2020-01-02
71942     2020-01-03
71943     2020-01-04
71944     2020-01-05
             ...    
2046361   2021-10-11
2046362   2021-10-12
2046363   2021-10-13
2046364   2021-10-14
2046365   2021-10-15
Name: dates, Length: 2038518, dtype: datetime64[ns]

In [69]:
print(anotherDF['dates'].dtype)
print(airDF['Date of Last Change'].dtype)
print(anotherDF['fips'].dtype)
print(airDF['fips'].dtype)

datetime64[ns]
datetime64[ns]
object
object


Export

In [ ]:
exportPbz2('covidPollutionCensus.pbz2',newDF)

And explore...

In [ ]:
# print(newDF.shape)
# mask = newDF.isnull().any(axis=0)
# noNullDF = newDF.loc[:,~mask]
# print(noNullDF.shape)

# newDF['density']=newDF['latestTotalPopulation']/newDF['LND110210']

# # a = [x for x in newDF if x.startswith('ozone')]
# b = [x for x in newDF if (x.startswith('JHU')|x.startswith('NYT'))&(~x.endswith('missing'))&('Confirmed' in x)]
# # b.extend(a)
# b
# no2DF = newDF.loc[newDF['pollutant']=='no2',b+['Observation Count','Observation Percent','Arithmetic Mean',\
#                                                            '1st Max Value','Mean ugm3','fips','dates','density']]
# # ozoneDF.dropna(subset=a,inplace=True)


# # ozoneDF.dropna(subset=['Mean ugm3'],inplace=True)
# no2fipsDF = no2DF[(no2DF['dates']>='2020-03-11')&(no2DF['dates']<='2021-03-11')].groupby(by=['fips']).agg({'JHU_ConfirmedDeaths.data': 'max', 'NYT_ConfirmedDeaths.data': 'max',\
#                               'Mean ugm3': 'mean','density': 'max'})

# no2fipsDF.corr()

# def covidPollutantFipsCorr(df, pollutant, cols=[]):
#     if cols:
#         polDF =  df.loc[df['pollutant']==pollutant,cols+['fips']]
#     else:
#         polDF = df.loc[df['pollutant']==pollutant,:]
#     fipsDF  = polDF.groupby(by='fips').agg('mean')
#     return fipsDF.corr()

# dateFilterDF = newDF[(newDF['dates']>='2020-03-11')|(newDF['dates']<='2021-03-11')]

# NO2

# covidPollutantFipsCorr(dateFilterDF, 'no2', b[:5]+['Arithmetic Mean','density'])

# Lead

# covidPollutantFipsCorr(dateFilterDF, 'lead', b[:5]+['Arithmetic Mean','density'])

# Ozone

# covidPollutantFipsCorr(dateFilterDF, 'ozone', b[:5]+['Arithmetic Mean','density'])

# PM2.5

# covidPollutantFipsCorr(dateFilterDF, 'pm25', b[:5]+['Arithmetic Mean','density'])

# a = no2fipsDF['Arithmetic Mean'].quantile([0,0.25,0.5,0.75,1])
# b = no2fipsDF['JHU_ConfirmedDeaths.data'].quantile([0,0.25,0.5,0.75,1])

# print(a)
# print(b)

# corrs =  newDF.corr()

# numbDF = newDF.loc[:,newDF.dtypes!='object']
# corrs = numbDF.corr()

# # # newDF.to_csv('allTogetherNow.gz',index=False,compression='gzip')
# # with bz2.BZ2File('allTogetherNow.pbz2','wb') as f:
# #     pickle.dump(newDF,f)